In [148]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [149]:
data = pd.read_csv(r'C:/Users/Sakshee/Downloads/AustriaNew.csv')

In [150]:
data.head()

,Unnamed: 0,index,AT_load_actual_entsoe_power_statistics,AT_load_actual_entsoe_transparency,AT_price_day_ahead,AT_solar_generation_actual,AT_wind_onshore_generation_actual,AT_windspeed_10m,AT_temperature,AT_radiation_direct_horizontal,AT_radiation_diffuse_horizontal
0,0,87656,7035.0,6343.0,36.0,2.0,109.0,3.27,-4.179,1.5614,25.9639
1,1,87657,7394.0,6882.0,41.0,10.0,146.0,3.23,-2.807,7.5467,75.0443
2,2,87658,7770.0,6963.0,45.0,21.0,146.0,3.22,-1.315,17.2073,116.9171
3,3,87659,7820.0,7110.0,50.0,32.0,158.0,3.22,-0.051,35.4289,146.8923
4,4,87660,7694.0,7136.0,51.0,37.0,187.0,3.00,0.791,58.5238,151.4721


In [151]:
data.shape

(17488, 11)

In [152]:
Solar=data[['AT_solar_generation_actual', 'AT_windspeed_10m', 'AT_temperature', 'AT_radiation_direct_horizontal', 'AT_radiation_diffuse_horizontal']]
Solar.head()

,AT_solar_generation_actual,AT_windspeed_10m,AT_temperature,AT_radiation_direct_horizontal,AT_radiation_diffuse_horizontal
0,2.0,3.27,-4.179,1.5614,25.9639
1,10.0,3.23,-2.807,7.5467,75.0443
2,21.0,3.22,-1.315,17.2073,116.9171
3,32.0,3.22,-0.051,35.4289,146.8923
4,37.0,3.00,0.791,58.5238,151.4721


In [153]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
solarScaled = scaler.fit_transform(Solar.values)
solarScaled = pd.DataFrame(solarScaled, columns = Solar.columns)

solarScaled.head()

,AT_solar_generation_actual,AT_windspeed_10m,AT_temperature,AT_radiation_direct_horizontal,AT_radiation_diffuse_horizontal
0,0.003279,0.285083,0.178747,0.001814,0.064377
1,0.016393,0.280663,0.209402,0.008767,0.186070
2,0.034426,0.279558,0.242738,0.019991,0.289892
3,0.052459,0.279558,0.270980,0.041160,0.364214
4,0.060656,0.255249,0.289794,0.067990,0.375570


In [155]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df, alpha=0.05): 
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

cointegration_test(solarScaled)

Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
AT_solar_generation_actual ::  5802.32   > 60.0627   =>   True
AT_windspeed_10m ::  2966.65   > 40.1749   =>   True
AT_temperature ::  1486.69   > 24.2761   =>   True
AT_radiation_direct_horizontal ::  249.37    > 12.3212   =>   True
AT_radiation_diffuse_horizontal ::  4.67      > 4.1296    =>   True


In [156]:
nobs = 3498
df_train, df_test = solarScaled[0:-nobs], solarScaled[-nobs:]

print(df_train.shape)  
print(df_test.shape) 

(13990, 5)
(3498, 5)


In [157]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')
        
    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")    

In [158]:
for name, column in df_train.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "AT_solar_generation_actual" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -4.1576
 No. Lags Chosen       = 42
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0008. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "AT_windspeed_10m" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -10.7991
 No. Lags Chosen       = 42
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "AT_temperature" 
    -----------------------------------------------
 Null Hypothesis: Data has unit ro

In [159]:
df_differenced = df_train.diff().dropna()

In [160]:
for name, column in df_differenced.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "AT_solar_generation_actual" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -29.7686
 No. Lags Chosen       = 42
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "AT_windspeed_10m" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -25.7427
 No. Lags Chosen       = 42
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "AT_temperature" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root

In [161]:
model = VAR(df_differenced)
for i in [1,2,3,4,5,6,7,8,9]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

Lag Order = 1
AIC :  -40.331837299883894
BIC :  -40.315653524294426
FPE :  3.0486362950989435e-18
HQIC:  -40.32644932083825 

Lag Order = 2
AIC :  -41.73406807842926
BIC :  -41.704395983032256
FPE :  7.501092601102817e-19
HQIC:  -41.724189469520425 

Lag Order =

C:\Users\Sakshee\miniconda3\envs\tensorflow\lib\site-packages\statsmodels\tsa\base\tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


 3
AIC :  -41.90168392568615
BIC :  -41.85852183726479
FPE :  6.343513826262441e-19
HQIC:  -41.88731409830884 

Lag Order = 4
AIC :  -41.982888161786015
BIC :  -41.92623440679196
FPE :  5.848753820339347e-19
HQIC:  -41.96402652721504 

Lag Order = 5
AIC :  -42.04104624429124
BIC :  -41.97089914884456
FPE :  5.518303874831169e-19
HQIC:  -42.017692213681435 

Lag Order = 6
AIC :  -42.08553086762915
BIC :  -42.0018887575182
FPE :  5.2782042493469555e-19
HQIC:  -42.05768385201538 

Lag Order = 7
AIC :  -42.14223183728147
BIC :  -42.04509303796285
FPE :  4.987251650055859e-19
HQIC:  -42.109891247578574 

Lag Order = 8
AIC :  -42.181928081663614
BIC :  -42.07129091826204
FPE :  4.793154572482705e-19
HQIC:  -42.145093328666384 

Lag Order = 9
AIC :  -42.22749026196332
BIC :  -42.103353059271534
FPE :  4.579668537002128e-19
HQIC:  -42.18616075634647 



In [162]:
model_fitted = model.fit(4)
model_fitted.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 01, Aug, 2020
Time:                     23:15:30
--------------------------------------------------------------------
No. of Equations:         5.00000    BIC:                   -41.9262
Nobs:                     13985.0    HQIC:                  -41.9640
Log likelihood:           194451.    FPE:                5.84875e-19
AIC:                     -41.9829    Det(Omega_mle):     5.80504e-19
--------------------------------------------------------------------
Results for equation AT_solar_generation_actual
                                        coefficient       std. error           t-stat            prob
-----------------------------------------------------------------------------------------------------
const                                      0.000015         0.000127            0.117           0.907
L1.AT_solar_generation_actual              1.160262      

In [163]:
# Get the lag order
lag_order = model_fitted.k_ar
print(lag_order)  #> 4

# Input data for forecasting
forecast_input = df_differenced.values[-lag_order:]
forecast_input

4


array([[ 0.        , -0.00110497, -0.00688176,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.00889266,  0.00046249,  0.00430832],
       [ 0.03114754, -0.02209945,  0.01941639,  0.04944503,  0.09290328],
       [ 0.12786885, -0.00773481,  0.06088569,  0.1682386 ,  0.0480406 ]])

In [164]:
# Forecast
fc = model_fitted.forecast(y=forecast_input, steps=nobs)

In [165]:
df_forecast = pd.DataFrame(fc, columns = Solar.columns)
df_forecast

,AT_solar_generation_actual,AT_windspeed_10m,AT_temperature,AT_radiation_direct_horizontal,AT_radiation_diffuse_horizontal
0,0.170187,-0.001481,0.067501,0.208406,-0.004682
1,0.162376,0.009645,0.055642,0.175063,-0.004785
2,0.127935,0.008313,0.039988,0.120303,0.007194
3,0.074890,0.007964,0.027087,0.055410,0.015159
4,0.013592,0.004996,0.016969,-0.013388,0.018942
...,...,...,...,...,...
3493,0.000027,-0.000007,0.000035,0.000031,-0.000016
3494,0.000027,-0.000007,0.000035,0.000031,-0.000016
3495,0.000027,-0.000007,0.000035,0.000031,-0.000016
3496,0.000027,-0.000007,0.000035,0.000031,-0.000016


In [175]:
# inverting transformation
def invert_transformation(df_differenced, df_forecast):
    forecast = df_forecast.copy()
    columns = df_train.columns
    for col in columns:
        forecast[str(col)] = df_train[col].iloc[-1] + forecast[str(col)].cumsum()
    return forecast

output = invert_transformation(df_differenced, df_forecast)

In [176]:
output.head()

,AT_solar_generation_actual,AT_windspeed_10m,AT_temperature,AT_radiation_direct_horizontal,AT_radiation_diffuse_horizontal
0,0.329203,0.120066,0.690367,0.426552,0.140571
1,0.491579,0.129711,0.746009,0.601614,0.135786
2,0.619514,0.138024,0.785997,0.721917,0.142980
3,0.694404,0.145987,0.813084,0.777327,0.158139
4,0.707996,0.150984,0.830053,0.763939,0.177082


In [178]:
from statsmodels.tsa.stattools import acf

def forecast_accuracy(forecast, actual):
    mae = np.mean(np.abs(forecast - actual))    # MAE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    return({'mae': mae, 'rmse':rmse})

In [179]:
print('Forecast Accuracy of: AT_solar_generation_actual')
accuracy = forecast_accuracy(output['AT_solar_generation_actual'].values, df_test['AT_solar_generation_actual'].values)
accuracy

Forecast Accuracy of: AT_solar_generation_actual


{'mae': 0.31932542025715116, 'rmse': 0.34190349795859964}

In [180]:
print('\nForecast Accuracy of: AT_windspeed_10m')
accuracy = forecast_accuracy(output['AT_windspeed_10m'].values, df_test['AT_windspeed_10m'].values)
accuracy


Forecast Accuracy of: AT_windspeed_10m


{'mae': 0.12383556314154362, 'rmse': 0.17081588488376923}

In [184]:
print('\nForecast Accuracy of: AT_temperature')
accuracy = forecast_accuracy(output['AT_temperature'].values, df_test['AT_temperature'].values)
accuracy


Forecast Accuracy of: AT_temperature


{'mae': 0.3669333198391381, 'rmse': 0.42067239283429303}

In [182]:
print('\nForecast Accuracy of: AT_radiation_direct_horizontal')
accuracy = forecast_accuracy(output['AT_radiation_direct_horizontal'].values, df_test['AT_radiation_direct_horizontal'].values)
accuracy


Forecast Accuracy of: AT_radiation_direct_horizontal


{'mae': 0.41983552889470294, 'rmse': 0.43813183036411407}

In [183]:
print('\nForecast Accuracy of: AT_radiation_diffuse_horizontal')
accuracy = forecast_accuracy(output['AT_radiation_diffuse_horizontal'].values, df_test['AT_radiation_diffuse_horizontal'].values)
accuracy


Forecast Accuracy of: AT_radiation_diffuse_horizontal


{'mae': 0.15218961888937835, 'rmse': 0.22922421540496712}